In [1]:
import numpy as np
from numpy import array as arr
from tabulate import tabulate

In [2]:
import sys
import socket
if socket.gethostname() == 'Birks-iMac.fritz.box':
    sys.path.append('/Users/Birk/Documents/Programmieren/Python/Eigener Code')
else:
    sys.path.append('/Home/ab227/linux/Eigener Code')
from pap import *
from pap import _rundung, _istbool, _chi_quadrat_print

# pap.vergleichstabelle()-Entwicklung

In [3]:
def _rundung_einzel(wert, präzision):
    '''
    Die Rundung mit einem Array von Nachkommastellen  (präzisionen)  funktioniert deshalb, da in rekursiven
    for-Loops die Arrays in einzelne Elemente aufgebrochen werden, welche dann gerundet werden.
    
    
    Argumente (sollten schon durch  _rundung()  passend gemacht worden sein.)
    ---------
    werte : np.ndarray (number_like), number_like
    
    präzisionen : np.ndarray (int), int
    
    
    Output
    ------
    wert_gerundet : np.ndarray (number_like), number_like
        Mit Pythons  round()  auf nächste gerade Ziffer gerundet (symmetrisches Runden).
        Form und Elemente sonst gleich wie  werte.
    '''
    
    
    
    anzahl_dimensionen = np.ndim(wert)   # Entsprechend viele Rekursionen werden stattfinden.
    
    # Eigentliche Rundung
    if anzahl_dimensionen == 0:
        wert_gerundet = round(wert, präzision)
    
    # Aufrufen von  _rundung_einzel()  um in einem for-Loop die Elemente von werte abzuarbeiten.
    elif anzahl_dimensionen > 0:
        wert_form = np.shape(wert)
        wert_gerundet = np.zeros(wert_form)
        for i in range(wert_form[0]):
            wert_gerundet[i] = _rundung_einzel(wert[i], präzision[i])
            
    return wert_gerundet   # Die gesammelten gerundeten Werte werden als Ergebnis zur vorigen Funktion
                           # zurückgeschickt.

In [4]:
def _rundung(werte, präzisionen):
    '''
    Rundet  werte  auf ihre jeweilige Anzahl Nachkommastellen  (präzisionen).
    Im Gegensatz zu  np.round()  kann diese Funktion  werte  mit ganzen Arrays von Nachkommastellen runden.
    Für Details zum Rundungsprozess siehe  _rundung_einzel().
    
    
    Argumente
    ---------
    werte : np.ndarray (number_like), number_like
        Kann beliebige Form haben und auch Einzelwert sein.
    
    präzisionen : np.ndarray (int), int
        Muss gleiche Form wie  werte  haben oder int sein.
    
    
    Output
    ------
    "wert_gerundet" : np.ndarray (number_like), number_like
        Mit Pythons  round()  auf nächste gerade Ziffer gerundet (symmetrisches Runden).
        Form und Elemente sonst gleich wie  werte.
    '''
    
    
    
    # Überprüfung und Anpassung der Argumente
    präzisionen = arr(präzisionen)  # Falls Einzelwert
    if präzisionen.dtype != int:
        print('präzisionen darf nur Integers enthalten!')
        print('Die Anzahl Stellen, auf die gerundet wird, kann nur ganzzahlig sein.')
    
    if np.shape(werte) != np.shape(präzisionen):
        if np.ndim(präzisionen) == 0:   # Falls  präzisionen  Einzelwert, dann wird
                                        # daraus ein Array derselben Form wie  werte  gemacht.
            präzisionen = np.full(np.shape(werte), präzisionen)
        else:
            print('werte und präzisionen müssen die gleiche np.shape haben!')
            print('np.shape(werte) =', np.shape(werte))
            print('np.shape(präzisionen) =', np.shape(präzisionen))
    
    
    # Eigentlicher Rundungsprozess
    return _rundung_einzel(werte, präzisionen)

In [5]:
def _negativ_wird_null(array):
    '''
    "Rampenfunktion": Die Werte von array, die negativ sind, werden durch 0 ersetzt.
    
    
    Argument
    --------
    array : np.ndarray (number_like)
    '''
    
    
    nullen = np.full(np.shape(array), 0)
    return np.where(array > 0, array, nullen)

In [6]:
def vergleichstabelle(werte:np.ndarray, einheit = '', faktor = 1):
    # Vorbearbeitung der Werte
    if np.shape(werte) == (4,):   
        werte = np.array(werte, ndmin = 2).T
    werte = werte * faktor   # Umrechnung auf gewünschte Größenordnung bzw. Einheit
    werte[1::2] = np.abs(werte[1::2])      # Keine negativen Fehlerwerte
    theo, theo_fehler, ex, ex_fehler = werte
    anzahl_vergleiche = np.shape(werte)[1]
    
    
    # Ausrechnen der Abweichungen
    abweichung_abs = ex - theo
    abweichung_abs_fehler = summen_fehler(arr([theo_fehler, ex_fehler]))
    abweichung_rel = abweichung_abs / theo * 100   # [%]
    abweichung_sig = np.abs(abweichung_abs / abweichung_abs_fehler)
    
    
    # Rundung der Werte
    ## Bestimmung der Nachkommastellen
    nachkommastellen = []
    listenzähler = 0
    for fehler_liste in [theo_fehler, ex_fehler, abweichung_abs_fehler, 
                         np.abs(abweichung_rel), abweichung_sig]:   # Alle Werte müssen > 0 sein.
        größenordnung_liste = arr((np.floor(np.log10(fehler_liste))), dtype = 'float64')
        signifikante_stellen = np.ones(anzahl_vergleiche)   # Normale Präzision: 1 sig. Stelle
        
        for i in range(anzahl_vergleiche):   # Wenn erste Ziffer klein, dann 2 sig. Stellen
            if listenzähler >= 3:   # rel. und sig. Abweichung bekommen immer 2 sig. Stellen
                signifikante_stellen[i] = 2
            elif fehler_liste[i] / 10 ** größenordnung_liste[i] < 4.0:  # Größe der ersten Ziffer
                signifikante_stellen[i] = 2        
        
        nachkommastellen_liste = -größenordnung_liste + signifikante_stellen - 1    
        nachkommastellen.append(nachkommastellen_liste)
        if listenzähler < 3:   # Die Nachkommastellen der Fehler-Werte müssen ein zweites Mal 
            nachkommastellen.append(nachkommastellen_liste)  # hinzugefügt werden für die "Hauptwerte"
        listenzähler +=1
    nachkommastellen = np.array(nachkommastellen, dtype = int)
    #print(nachkommastellen)
    #print('form_nachkomma:', np.shape(nachkommastellen))
    
    ## Rundung aller Werte
    alle_werte = arr([theo, theo_fehler, ex, ex_fehler, abweichung_abs, abweichung_abs_fehler,
                      abweichung_rel, abweichung_sig])
    #print('form_werte:', np.shape(alle_werte))
    alle_werte_gerundet = _rundung(alle_werte, nachkommastellen)
    #print(alle_werte_gerundet)
    
    #print(nachkommastellen)
    nachkommastellen = _negativ_wird_null(nachkommastellen)
    # Um Fehler bei der String-Formatierung zu vermeiden.
    #print(nachkommastellen)
    #alle_werte_strings = arr([[]])
    
    # Darstellung der Werte
    alle_vergleiche = []
    for i in range(anzahl_vergleiche):
        #print(alle_werte_gerundet[0][i])
        #print(alle_werte_gerundet[1][i])
        #print(nachkommastellen[0][i])
        vergleich = ['{:.{prec}f} +/- {:.{prec}f}'.format(alle_werte_gerundet[0][i], alle_werte_gerundet[1][i],
                                                          prec = nachkommastellen[0][i]),
                     '{:.{prec}f} +/- {:.{prec}f}'.format(alle_werte_gerundet[2][i], alle_werte_gerundet[3][i],
                                                          prec = nachkommastellen[2][i]),
                     '{:.{prec}f} +/- {:.{prec}f}'.format(alle_werte_gerundet[4][i], alle_werte_gerundet[5][i],
                                                          prec = nachkommastellen[4][i]),
                     '{:.{prec}f} %'.format(alle_werte_gerundet[6][i], prec = nachkommastellen[6][i]),
                     '{:.{prec}f} σ'.format(alle_werte_gerundet[7][i], prec = nachkommastellen[7][i])]
        
        alle_vergleiche.append(vergleich)
        
    alle_vergleiche = arr(alle_vergleiche)
    print(alle_vergleiche.T)
    print('')
    
    
    '''
    werte_aufzählung = np.arange(anzahl_vergleiche)   # [0, ..., "anzahl_vergleiche" - 1]
    print([f'{theo[i]} +/- {theo_fehler[i]}' for i in werte_aufzählung])
    print([f'{ex[i]} +/- {ex_fehler[i]}' for i in werte_aufzählung])
    print([f'{abweichung_abs[i]} +/- {abweichung_abs_fehler[i]}'  for i in werte_aufzählung])
    print([f'{abweichung_rel[i]} %' for i in werte_aufzählung])
    print([f'{abweichung_sig[i]}' for i in werte_aufzählung])
    print('')
    '''

    '''
    # Rundung entsprechend der signifikante Stelle oder der eigenen Vorgabe
    wertepaar = _rundung(werte, nachkommastellen)
    if nachkommastellen <= 0:   # Um Fehler bei der String-Formatierung zu vermeiden.
        nachkommastellen = 0
    
    
    # Print des Ergebnis-Strings
    if len(werte) == 1:
        print(titel + ': {:.{prec}f} {}{}'
              .format(*wertepaar, einheit, rel_fehler_string, prec = nachkommastellen))
    elif len(werte) == 2:
        print(titel + ': {:.{prec}f} +/- {:.{prec}f} {}{}'
              .format(*wertepaar, einheit, rel_fehler_string, prec = nachkommastellen))
    elif len(werte) == 3:
        print(titel + ': {:.{prec}f} +/- {:.{prec}f}(sys) +/- {:.{prec}f}(stat) {}{}'
              .format(*wertepaar, einheit, rel_fehler_string, prec = nachkommastellen))
    ''';




## Tests

In [7]:
# Test der Funktion:

# Test mit normalen Werten
vergleichstabelle(arr([0.123, 0.012, 1.098, 0.0987]))

# Tests mit negativen Werten und negativen Fehlern
vergleichstabelle(arr([0.123, -0.012, 1.098, 0.0987]))
vergleichstabelle(arr([-0.123, 0.012, -1.098, 0.0987]))
vergleichstabelle(arr([0.123, 0.012, -1.098, 0.0987]))


# Test mit zwei Vergleichen
vergleichstabelle(arr([[0.123, 400 / 7], 
                       [0.012, 23 / 7], 
                       [1.098, 337 / 7], 
                       [0.987, 29 / 7]]))

# Test der Rundung
vergleichstabelle(arr([[284.3858, 0.07538], 
                       [40.0000, 0.00400], 
                       [317.3048, 0.07536], 
                       [39.9999, 0.00399]]))

[['0.123 +/- 0.012']
 ['1.10 +/- 0.10']
 ['0.98 +/- 0.10']
 ['790 %']
 ['9.8 σ']]

[['0.123 +/- 0.012']
 ['1.10 +/- 0.10']
 ['0.98 +/- 0.10']
 ['790 %']
 ['9.8 σ']]

[['-0.123 +/- 0.012']
 ['-1.10 +/- 0.10']
 ['-0.98 +/- 0.10']
 ['790 %']
 ['9.8 σ']]

[['0.123 +/- 0.012']
 ['-1.10 +/- 0.10']
 ['-1.22 +/- 0.10']
 ['-990 %']
 ['12 σ']]

[['0.123 +/- 0.012' '57.1 +/- 3.3']
 ['1.1 +/- 1.0' '48 +/- 4']
 ['1.0 +/- 1.0' '-9 +/- 5']
 ['790 %' '-16 %']
 ['0.99 σ' '1.7 σ']]

[['280 +/- 40' '0.075 +/- 0.004']
 ['317 +/- 40' '0.0754 +/- 0.0040']
 ['30 +/- 60' '-0.000 +/- 0.006']
 ['12 %' '-0.027 %']
 ['0.58 σ' '0.0035 σ']]

